# Domain pruning in holocleansession 
In this notebook we will show the mechanism of domain pruning and how it works. To start we import the necessary modules 

In [1]:
import sys
import os
import time
sys.path.append('../')
from holoclean import holocleansession , dataset , dataengine
from holoclean.utils import domainpruning
from holoclean.featurization import dcfeaturizer,dc_featurizer

After import file we have to indicate the driver file path for mysql connector and initialize the holocleansession with it.

In [2]:
my_path = os.path.abspath(os.path.dirname('__file__'))
path = os.path.join(my_path, "../holoclean/mysql-connector-java-5.1.44-bin.jar")
holoclean_se=holocleansession.HolocleanSession(path)

We have to create dataset object to create specific object that track the project tables in the database and after that we ingest the csv to spark and database.

In [3]:
ds=dataset.Dataset()  
d=dataengine.Dataengine("metadb-config.txt",'datadb-config.txt',ds,holoclean_se.return_sqlcontext())
holoclean_se.set_dataengine(d)
df=d.ingest_spark('100.csv',holoclean_se.returnspark_session())

index


In here based on the denial constraints we divide the cells into two sets cells that we know their values are clean and cells that have some violations.

In [4]:
dcCode=['t1&t2&EQ(t1.city,t2.city)&EQ(t1.temp,t2.temp)&IQ(t1.tempType,t2.tempType)']

dk_cells,clean_cells=holoclean_se._error_detection(dcCode,df)

ind
ind


In here we try to prune the domain the default value for threshold is 0.5

In [5]:
can=holoclean_se._domain_prunnig()
can.show(can.count())

attr
+--------+-----------+
|    attr|      value|
+--------+-----------+
|    date|   18000101|
|    date|   18000102|
|    date|   18000103|
|    date|   18000104|
|    date|   18000105|
|    date|   18000106|
|    date|   18000107|
|    date|   18000108|
|    date|   18000109|
|    date|   18000110|
|    date|   18000111|
|    date|   18000112|
|    date|   18000113|
|    date|   18000114|
|    date|   18000115|
|    date|   18000116|
|    date|   18000117|
|    date|   18000118|
|    date|   18000119|
|    date|   18000120|
|    city|EZE00100082|
|    city|ITE00100554|
|tempType|       TMAX|
|tempType|       TMIN|
|    temp|       -130|
|    temp|        -46|
|    temp|         10|
|    temp|         13|
|    temp|         23|
|    temp|         29|
|    temp|         31|
|    temp|         41|
|    temp|         46|
+--------+-----------+



In here we can call the table from mysql database too

In [6]:
d_in_db=d.get_table_spark("D")
d_in_db.show(d_in_db.count())

+--------+-----------+
|    attr|      value|
+--------+-----------+
|    date|   18000117|
|    date|   18000118|
|    date|   18000119|
|    date|   18000120|
|    city|EZE00100082|
|    city|ITE00100554|
|tempType|       TMAX|
|tempType|       TMIN|
|    date|   18000101|
|    date|   18000102|
|    date|   18000103|
|    date|   18000104|
|    date|   18000105|
|    date|   18000106|
|    date|   18000107|
|    date|   18000108|
|    date|   18000109|
|    date|   18000110|
|    date|   18000111|
|    date|   18000112|
|    date|   18000113|
|    date|   18000114|
|    date|   18000115|
|    date|   18000116|
|    temp|       -130|
|    temp|        -46|
|    temp|         10|
|    temp|         13|
|    temp|         23|
|    temp|         29|
|    temp|         31|
|    temp|         41|
|    temp|         46|
+--------+-----------+



Next create the dc-featurizer with the domain that we pruned

In [7]:
dc_f=dc_featurizer.DCFeaturizer(df,dcCode,d)

start_time_dd = time.time()
asd=dc_f.table_featurizer_pruning()
finish_time_dd=time.time()
mysqlv=d.get_table_spark("dc_f1")
print(mysqlv.count())
print(mysqlv.show())

1168
+--------+------------+--------+------+--------------------+
|rv_index|assigned_val| rv_attr|tup_id|             DC_name|
+--------+------------+--------+------+--------------------+
|       1|        TMIN|tempType|     1|table1.city=table...|
|       1|         -46|    temp|    12|table1.city=table...|
|       1|          13|    temp|    27|table1.city=table...|
|       1|          10|    temp|    32|table1.city=table...|
|       1|          23|    temp|    42|table1.city=table...|
|       1|          31|    temp|    47|table1.city=table...|
|       1| EZE00100082|    city|    50|table1.city=table...|
|       1|          41|    temp|    52|table1.city=table...|
|       1|          29|    temp|    57|table1.city=table...|
|       1|          13|    temp|    62|table1.city=table...|
|       1|          23|    temp|    67|table1.city=table...|
|       1|          41|    temp|    72|table1.city=table...|
|       1|          41|    temp|    77|table1.city=table...|
|       1|         

In [8]:
print("--- %s seconds ---" % (finish_time_dd-start_time_dd))

--- 0.155375957489 seconds ---
